<a href="https://colab.research.google.com/github/saiteja-1919/Chest-x-ray-classification/blob/main/dl_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import files
print("📤 Please upload the CHEST DATA.zip file")
uploaded = files.upload()

zip_file = list(uploaded.keys())[0]
extract_dir = "/content/chest_data"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"✅ Dataset extracted to {extract_dir}")


base_dir = os.path.join(extract_dir, "dataset")

img_size = (128, 128)
batch_size = 32
epochs = 3

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.5)

client1_data = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='trai-ning',
    seed=42
)

client2_data = datagen.flow_from_directory(
    base_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    seed=42
)

NUM_CLASSES = len(client1_data.class_indices)
print(f"\n🔹 Number of classes: {NUM_CLASSES}")

def create_cnn():
    model = models.Sequential([
        layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

client1_model = create_cnn()
client2_model = create_cnn()


print("\n=== Initial Weights ===")
print("Client 1 Conv1 first filter (first 5 values):", client1_model.get_weights()[0][0,0,0,:5])
print("Client 2 Conv1 first filter (first 5 values):", client2_model.get_weights()[0][0,0,0,:5])
print("Client 1 Output layer weights (first 5 values):", client1_model.get_weights()[-2].flatten()[:5])
print("Client 2 Output layer weights (first 5 values):", client2_model.get_weights()[-2].flatten()[:5])


print("\n🚀 Training Client 1...")
client1_model.fit(client1_data, epochs=epochs, verbose=1)

print("\n🚀 Training Client 2...")
client2_model.fit(client2_data, epochs=epochs, verbose=1)


weights_client1 = client1_model.get_weights()
weights_client2 = client2_model.get_weights()

print("\n=== Weights After Local Training ===")
print("Client 1 Conv1 first filter (first 5 values):", weights_client1[0][0,0,0,:5])
print("Client 2 Conv1 first filter (first 5 values):", weights_client2[0][0,0,0,:5])
print("Client 1 Output layer weights (first 5 values):", weights_client1[-2].flatten()[:5])
print("Client 2 Output layer weights (first 5 values):", weights_client2[-2].flatten()[:5])


def fed_avg(weights_list):
    avg_weights = []
    for w in zip(*weights_list):
        avg_weights.append(np.mean(w, axis=0))
    return avg_weights

new_global_weights = fed_avg([weights_client1, weights_client2])


global_model = create_cnn()
global_model.set_weights(new_global_weights)

print("\n=== Global Model Weights After FedAvg ===")
print("Global Conv1 first filter (first 5 values):", global_model.get_weights()[0][0,0,0,:5])
print("Global Output layer weights (first 5 values):", global_model.get_weights()[-2].flatten()[:5])


loss, acc = global_model.evaluate(client1_data, steps=client1_data.samples // batch_size, verbose=0)
print(f"\n✅ Global model accuracy on Client 1 data: {acc:.4f}")


📤 Please upload the CHEST DATA.zip file


Saving CHEST DATA.zip to CHEST DATA (3).zip
✅ Dataset extracted to /content/chest_data
Found 48 images belonging to 2 classes.
Found 46 images belonging to 2 classes.

🔹 Number of classes: 2

=== Initial Weights ===
Client 1 Conv1 first filter (first 5 values): [ 0.03892985  0.08391857 -0.01321472 -0.05159143 -0.04962701]
Client 2 Conv1 first filter (first 5 values): [ 0.07290539  0.12192315 -0.13401586 -0.04948205 -0.0211093 ]
Client 1 Output layer weights (first 5 values): [-0.26589087 -0.06606904 -0.20294333  0.26775855  0.16507003]
Client 2 Output layer weights (first 5 values): [-0.27908927  0.24084508  0.05968916 -0.07617655 -0.01251081]

🚀 Training Client 1...
Epoch 1/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.4583 - loss: 1.5689
Epoch 2/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 849ms/step - accuracy: 0.6528 - loss: 1.6235
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step - accuracy: 0.7778 - loss: 0.4963

🚀 Training Client 2...
Epoch 1/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accura